In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 30*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240330011250
0	Validation losses: 0.4341, 21.1849, 21.6190	Best Loss: 21.6190 (0)	Accuracy: 83.78%
1	Validation losses: 0.3841, 11.4681, 11.8522	Best Loss: 11.8522 (0)	Accuracy: 86.76%
2	Validation losses: 0.4000, 4.2460, 4.6460	Best Loss: 4.6460 (0)	Accuracy: 85.30%
3	Validation losses: 0.4565, 1.1903, 1.6467	Best Loss: 1.6467 (0)	Accuracy: 83.66%
4	Validation losses: 0.4157, 0.4089, 0.8246	Best Loss: 0.8246 (0)	Accuracy: 84.62%
5	Validation losses: 0.5062, 0.2713, 0.7775	Best Loss: 0.7775 (0)	Accuracy: 81.26%
6	Validation losses: 0.3852, 0.2437, 0.6289	Best Loss: 0.6289 (0)	Accuracy: 86.52%
7	Validation losses: 0.5891, 0.2330, 0.8221	Best Loss: 0.6289 (1)	Accuracy: 77.48%
8	Validation losses: 0.4345, 0.2314, 0.6659	Best Loss: 0.6289 (2)	Accuracy: 84.74%
9	Validation losses: 0.4743, 0.2272, 0.7015	Best Loss: 0.6289 (3)	Accuracy: 83.20%
10	Validation losses: 0.5259, 0.2083, 0.7342	Best Loss: 0.6289 (4)	Accuracy: 82.02%
11	Validation losses: 0

2	Validation losses: 0.3769, 5.5109, 5.8878	Best Loss: 5.8878 (0)	Accuracy: 86.40%
3	Validation losses: 0.4221, 2.3828, 2.8049	Best Loss: 2.8049 (0)	Accuracy: 84.66%
4	Validation losses: 0.3830, 0.8951, 1.2781	Best Loss: 1.2781 (0)	Accuracy: 85.42%
5	Validation losses: 0.4922, 0.3815, 0.8737	Best Loss: 0.8737 (0)	Accuracy: 82.48%
6	Validation losses: 0.3561, 0.2456, 0.6017	Best Loss: 0.6017 (0)	Accuracy: 87.02%
7	Validation losses: 0.4792, 0.2122, 0.6914	Best Loss: 0.6017 (1)	Accuracy: 82.02%
8	Validation losses: 0.4153, 0.2061, 0.6213	Best Loss: 0.6017 (2)	Accuracy: 85.06%
9	Validation losses: 0.7091, 0.2029, 0.9120	Best Loss: 0.6017 (3)	Accuracy: 76.38%
10	Validation losses: 0.4397, 0.1998, 0.6395	Best Loss: 0.6017 (4)	Accuracy: 84.54%
11	Validation losses: 0.4186, 0.1961, 0.6147	Best Loss: 0.6017 (5)	Accuracy: 84.54%
12	Validation losses: 0.5623, 0.1863, 0.7486	Best Loss: 0.6017 (6)	Accuracy: 79.72%
13	Validation losses: 0.5007, 0.1834, 0.6840	Best Loss: 0.6017 (7)	Accuracy: 82.42%


3	Validation losses: 0.4100, 2.3814, 2.7913	Best Loss: 2.7913 (0)	Accuracy: 85.06%
4	Validation losses: 0.4178, 0.8935, 1.3112	Best Loss: 1.3112 (0)	Accuracy: 84.84%
5	Validation losses: 0.4214, 0.3815, 0.8029	Best Loss: 0.8029 (0)	Accuracy: 84.22%
6	Validation losses: 0.4454, 0.2436, 0.6890	Best Loss: 0.6890 (0)	Accuracy: 84.24%
7	Validation losses: 0.4610, 0.2135, 0.6745	Best Loss: 0.6745 (0)	Accuracy: 83.08%
8	Validation losses: 0.4080, 0.2047, 0.6127	Best Loss: 0.6127 (0)	Accuracy: 85.36%
9	Validation losses: 0.9749, 0.2001, 1.1750	Best Loss: 0.6127 (1)	Accuracy: 69.30%
10	Validation losses: 0.4985, 0.2025, 0.7010	Best Loss: 0.6127 (2)	Accuracy: 82.10%
11	Validation losses: 0.4803, 0.1973, 0.6776	Best Loss: 0.6127 (3)	Accuracy: 80.46%
12	Validation losses: 0.4988, 0.1948, 0.6937	Best Loss: 0.6127 (4)	Accuracy: 82.46%
13	Validation losses: 0.4736, 0.1826, 0.6563	Best Loss: 0.6127 (5)	Accuracy: 83.82%
14	Validation losses: 0.4483, 0.1980, 0.6463	Best Loss: 0.6127 (6)	Accuracy: 84.40%

36	Validation losses: 0.3582, 0.1253, 0.4835	Best Loss: 0.4835 (0)	Accuracy: 87.18%
37	Validation losses: 0.3582, 0.1242, 0.4825	Best Loss: 0.4825 (0)	Accuracy: 87.08%
38	Validation losses: 0.3483, 0.1237, 0.4720	Best Loss: 0.4720 (0)	Accuracy: 87.36%
39	Validation losses: 0.3454, 0.1234, 0.4688	Best Loss: 0.4688 (0)	Accuracy: 87.48%
40	Validation losses: 0.3390, 0.1232, 0.4622	Best Loss: 0.4622 (0)	Accuracy: 87.54%
41	Validation losses: 0.3407, 0.1230, 0.4637	Best Loss: 0.4622 (1)	Accuracy: 87.72%
42	Validation losses: 0.3438, 0.1232, 0.4670	Best Loss: 0.4622 (2)	Accuracy: 87.24%
43	Validation losses: 0.3467, 0.1237, 0.4704	Best Loss: 0.4622 (3)	Accuracy: 87.48%
44	Validation losses: 0.3545, 0.1240, 0.4785	Best Loss: 0.4622 (4)	Accuracy: 87.14%
45	Validation losses: 0.3605, 0.1249, 0.4853	Best Loss: 0.4622 (5)	Accuracy: 87.50%
46	Validation losses: 0.3520, 0.1249, 0.4768	Best Loss: 0.4622 (6)	Accuracy: 88.04%
47	Validation losses: 0.3495, 0.1251, 0.4745	Best Loss: 0.4622 (7)	Accuracy:

3	Validation losses: 0.3791, 3.2271, 3.6063	Best Loss: 3.6063 (0)	Accuracy: 85.96%
4	Validation losses: 0.3735, 1.6792, 2.0527	Best Loss: 2.0527 (0)	Accuracy: 86.28%
5	Validation losses: 0.3848, 0.8058, 1.1906	Best Loss: 1.1906 (0)	Accuracy: 85.76%
6	Validation losses: 0.4066, 0.4036, 0.8103	Best Loss: 0.8103 (0)	Accuracy: 85.28%
7	Validation losses: 0.4213, 0.2518, 0.6731	Best Loss: 0.6731 (0)	Accuracy: 84.32%
8	Validation losses: 0.3793, 0.2013, 0.5806	Best Loss: 0.5806 (0)	Accuracy: 86.16%
9	Validation losses: 0.8614, 0.1880, 1.0494	Best Loss: 0.5806 (1)	Accuracy: 73.22%
10	Validation losses: 0.4668, 0.1843, 0.6511	Best Loss: 0.5806 (2)	Accuracy: 83.46%
11	Validation losses: 0.4564, 0.1844, 0.6407	Best Loss: 0.5806 (3)	Accuracy: 81.52%
12	Validation losses: 0.4192, 0.1928, 0.6120	Best Loss: 0.5806 (4)	Accuracy: 84.54%
13	Validation losses: 0.4407, 0.1789, 0.6196	Best Loss: 0.5806 (5)	Accuracy: 84.08%
14	Validation losses: 0.4153, 0.1788, 0.5940	Best Loss: 0.5806 (6)	Accuracy: 85.40%

...

12	Validation losses: 0.4169, 0.1122, 0.5291	Best Loss: 0.4112 (6)	Accuracy: 86.30%
13	Validation losses: 0.3338, 0.1126, 0.4464	Best Loss: 0.4112 (7)	Accuracy: 88.24%
14	Validation losses: 0.2983, 0.1132, 0.4115	Best Loss: 0.4112 (8)	Accuracy: 89.32%
15	Validation losses: 0.3324, 0.1140, 0.4464	Best Loss: 0.4112 (9)	Accuracy: 88.76%
16	Validation losses: 0.3316, 0.1147, 0.4463	Best Loss: 0.4112 (10)	Accuracy: 89.12%
17	Validation losses: 0.3215, 0.1153, 0.4368	Best Loss: 0.4112 (11)	Accuracy: 89.16%
18	Validation losses: 0.3198, 0.1161, 0.4359	Best Loss: 0.4112 (12)	Accuracy: 89.30%
19	Validation losses: 0.3479, 0.1168, 0.4647	Best Loss: 0.4112 (13)	Accuracy: 87.82%
20	Validation losses: 0.3658, 0.1176, 0.4835	Best Loss: 0.4112 (14)	Accuracy: 88.02%
21	Validation losses: 0.3244, 0.1182, 0.4426	Best Loss: 0.4112 (15)	Accuracy: 89.22%
22	Validation losses: 0.3264, 0.1182, 0.4446	Best Loss: 0.4112 (16)	Accuracy: 89.86%
23	Validation losses: 0.3161, 0.1184, 0.4345	Best Loss: 0.4112 (17)	A

6	Validation losses: 0.2978, 0.0705, 0.3682	Best Loss: 0.3682 (0)	Accuracy: 89.14%
7	Validation losses: 0.3378, 0.0708, 0.4086	Best Loss: 0.3682 (1)	Accuracy: 87.62%
8	Validation losses: 0.2953, 0.0712, 0.3665	Best Loss: 0.3665 (0)	Accuracy: 89.32%
9	Validation losses: 0.3187, 0.0717, 0.3905	Best Loss: 0.3665 (1)	Accuracy: 89.08%
10	Validation losses: 0.2984, 0.0723, 0.3707	Best Loss: 0.3665 (2)	Accuracy: 89.46%
11	Validation losses: 0.3151, 0.0728, 0.3879	Best Loss: 0.3665 (3)	Accuracy: 89.00%
12	Validation losses: 0.4161, 0.0735, 0.4897	Best Loss: 0.3665 (4)	Accuracy: 86.36%
13	Validation losses: 0.3589, 0.0742, 0.4331	Best Loss: 0.3665 (5)	Accuracy: 87.78%
14	Validation losses: 0.3046, 0.0750, 0.3796	Best Loss: 0.3665 (6)	Accuracy: 89.84%
15	Validation losses: 0.3344, 0.0758, 0.4102	Best Loss: 0.3665 (7)	Accuracy: 88.76%
16	Validation losses: 0.3160, 0.0767, 0.3927	Best Loss: 0.3665 (8)	Accuracy: 89.44%
17	Validation losses: 0.3370, 0.0776, 0.4146	Best Loss: 0.3665 (9)	Accuracy: 88.

0	Validation losses: 0.4026, 0.0438, 0.4464	Best Loss: 0.4464 (0)	Accuracy: 84.94%
1	Validation losses: 0.3510, 0.0438, 0.3948	Best Loss: 0.3948 (0)	Accuracy: 87.34%
2	Validation losses: 0.3660, 0.0439, 0.4099	Best Loss: 0.3948 (1)	Accuracy: 86.74%
3	Validation losses: 0.3416, 0.0441, 0.3857	Best Loss: 0.3857 (0)	Accuracy: 87.38%
4	Validation losses: 0.3162, 0.0443, 0.3604	Best Loss: 0.3604 (0)	Accuracy: 88.36%
5	Validation losses: 0.3934, 0.0445, 0.4380	Best Loss: 0.3604 (1)	Accuracy: 85.80%
6	Validation losses: 0.2942, 0.0448, 0.3390	Best Loss: 0.3390 (0)	Accuracy: 88.98%
7	Validation losses: 0.3253, 0.0451, 0.3705	Best Loss: 0.3390 (1)	Accuracy: 88.20%
8	Validation losses: 0.3011, 0.0455, 0.3466	Best Loss: 0.3390 (2)	Accuracy: 89.02%
9	Validation losses: 0.3634, 0.0460, 0.4094	Best Loss: 0.3390 (3)	Accuracy: 87.62%
10	Validation losses: 0.3074, 0.0464, 0.3538	Best Loss: 0.3390 (4)	Accuracy: 89.50%
11	Validation losses: 0.3225, 0.0469, 0.3694	Best Loss: 0.3390 (5)	Accuracy: 89.12%
12

4	Validation losses: 0.3306, 0.0280, 0.3586	Best Loss: 0.3586 (0)	Accuracy: 87.40%
5	Validation losses: 0.3835, 0.0282, 0.4117	Best Loss: 0.3586 (1)	Accuracy: 86.42%
6	Validation losses: 0.2964, 0.0284, 0.3247	Best Loss: 0.3247 (0)	Accuracy: 89.12%
7	Validation losses: 0.3403, 0.0286, 0.3690	Best Loss: 0.3247 (1)	Accuracy: 87.68%
8	Validation losses: 0.3080, 0.0289, 0.3370	Best Loss: 0.3247 (2)	Accuracy: 88.72%
9	Validation losses: 0.3277, 0.0292, 0.3569	Best Loss: 0.3247 (3)	Accuracy: 89.06%
10	Validation losses: 0.3041, 0.0296, 0.3336	Best Loss: 0.3247 (4)	Accuracy: 89.30%
11	Validation losses: 0.2993, 0.0300, 0.3292	Best Loss: 0.3247 (5)	Accuracy: 89.20%
12	Validation losses: 0.3636, 0.0304, 0.3940	Best Loss: 0.3247 (6)	Accuracy: 87.96%
13	Validation losses: 0.3312, 0.0308, 0.3621	Best Loss: 0.3247 (7)	Accuracy: 88.10%
14	Validation losses: 0.3015, 0.0313, 0.3328	Best Loss: 0.3247 (8)	Accuracy: 89.40%
15	Validation losses: 0.3360, 0.0319, 0.3678	Best Loss: 0.3247 (9)	Accuracy: 88.36

Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.9399988651%
